In [15]:
JULIA = "/usr/local/bin/julia"

In [16]:
import subprocess

def find_juliac(julia):
    snippet = 'normpath(Base.find_package("PackageCompiler"), "..", "..", "juliac.jl")'
    out = subprocess.check_output(
        [JULIA, "--startup-file=no", "-E", snippet],
        encoding='utf-8'
    )
    return out.splitlines()[0][1:-1]

In [17]:
JULIAC_PATH = find_juliac(JULIA)

print(JULIAC_PATH)

/Users/jvankerschaver/.julia/packages/PackageCompiler/jBqfm/juliac.jl


In [18]:
julia_snippet = """
Base.@ccallable function nbmin(buffer::Ptr{Cdouble},
                               filter_size::Int,
                               return_value::Ptr{Cdouble},
                               user_data::Ptr{Cvoid})::Cint

    minval = Base.Inf
    for i = 1:filter_size
        value = unsafe_load(buffer, i)
        if value < minval
            minval = value
        end
    end

    unsafe_store!(return_value, 1, minval)
        
    return 1  # success
end

"""

In [25]:
julia_snippet = """
Base.@ccallable function nbmin(buffer::Ptr{Cdouble},
                               filter_size::Int,
                               return_value::Ptr{Cdouble},
                               user_data::Ptr{Cvoid})::Cint


    value = unsafe_load(buffer, 1)
    unsafe_store!(return_value, 1, value)
        
    return 1  # success
end

"""

In [26]:


import os

def invoke_juliac(fname, builddir=None):
    cmd = [JULIA, JULIAC_PATH, '-vas', fname]
    if builddir:
        cmd += ['-d', builddir]
    subprocess.check_call(cmd)

In [27]:
# invoke_juliac("/Users/jvankerschaver/tmp/xyz.jl")

In [28]:
import uuid

def generate_temp_folder():
    fname = "juliac-{}".format(uuid.uuid4().hex)
    os.makedirs(fname)
    return fname

def compile_snippet(code):
    builddir = generate_temp_folder()
    
    code_fname = os.path.join(builddir, "code.jl")
    with open(code_fname, "w") as fp:
        fp.write(code)
        
    invoke_juliac(code_fname, builddir)
    lib_fname = os.path.abspath(os.path.join(builddir, "code.dylib"))
    return lib_fname

In [29]:
image = compile_snippet(julia_snippet)

In [30]:
image

'/Users/jvankerschaver/tmp/juliac-e767934a23dd4df8bb233f1a588aa642/code.dylib'

In [2]:
import ctypes

In [4]:
image = '/Users/jvankerschaver/tmp/juliac-b363fb048d5a4def953195da1bd89e03/code.dylib'

In [5]:
julia_dll = ctypes.CDLL(image, ctypes.RTLD_GLOBAL)

try:
    jl_init_with_image = julia_dll.jl_init_with_image
except AttributeError:
    jl_init_with_image = julia_dll.jl_init_with_image__threading
    
jl_init_with_image.argtypes = [ctypes.c_char_p, ctypes.c_char_p]

jl_init_with_image(None, image.encode("utf-8"))

44131840

In [6]:
py_buffer = [1.0, 0.3, 2.5]
buffer = (ctypes.c_double * len(py_buffer))(*py_buffer)

filter_size = len(py_buffer)

py_result = [0]
result = (ctypes.c_double * len(py_result))(*py_result)

In [8]:
buffer

In [7]:
# Call directly
julia_dll.nbmin.res_type = ctypes.c_int
julia_dll.nbmin.argtypes = (
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_void_p
)

In [9]:
julia_dll.nbmin(ctypes.byref(buffer), filter_size, ctypes.byref(result), None)

ArgumentError: argument 1: <class 'TypeError'>: expected LP_c_double instance instead of pointer to c_double_Array_3